In [1]:
!pip install rtree

import os
import json
# import yaml
import rtree
import fiona

from scipy.spatial import distance
import geopandas as gpd
import pandas as pd
from scipy.spatial.distance import cdist
from os import path as op
from ipyleaflet import GeoJSON
from shapely.geometry import Point
from shapely.ops import nearest_points
import numpy as np
from scipy import stats

/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


Point to input files and variables

In [2]:
#Point to input files and other variables
sample_clu_path = '/Users/sorenrundquist/Documents/ParcelCheck/test_clu.geojson'
sample_parcels_path = '/Users/sorenrundquist/Documents/ParcelCheck/test_parcel_id.geojson'
output_results_path = '/Users/sorenrundquist/Documents/ParcelCheck/'
union_results_path = '/Users/sorenrundquist/Documents/ParcelCheck/parcel_CLU_union.geojson'
parcel_id = 'parID'
clu_id = 'cluID'
parcel_area = 'parArea'
clu_area = 'cluArea'
union_area = 'uniArea'
expression = 'uniArea >= 5'
clu_area_minus_uni_area = 'difOverlap'
conus_epsg = 'EPSG:5070'
sample_parcels_path_reproj = output_results_path +"test_parcel_id_reproj.geojson"
sample_clu_path_reproj = output_results_path +"test_clu_reproj.geojson"


In [4]:
#check existing projection and reproject
def get_geojson_epsg(sample_parcels_path):
    # Open the GeoJSON file
    with fiona.open(sample_parcels_path) as src:
        # Get the CRS (coordinate reference system) of the GeoJSON file
        crs = src.crs
    
    # Extract the EPSG code from the CRS
    if 'init' in crs:
        epsg_code = int(crs['init'].split(':')[1])
    elif 'AUTHORITY' in crs:
        epsg_code = int(crs['AUTHORITY'][1])
    else:
        epsg_code = None
    
    return epsg_code

epsg_code_par = get_geojson_epsg(sample_parcels_path)
print("Parcel ID EPSG code:", epsg_code_par)

def get_geojson_epsg(sample_clu_path):
    # Open the GeoJSON file
    with fiona.open(sample_clu_path) as src:
        # Get the CRS (coordinate reference system) of the GeoJSON file
        crs = src.crs
    
    # Extract the EPSG code from the CRS
    if 'init' in crs:
        epsg_code = int(crs['init'].split(':')[1])
    elif 'AUTHORITY' in crs:
        epsg_code = int(crs['AUTHORITY'][1])
    else:
        epsg_code = None
    
    return epsg_code

# Example usage:
# geojson_path = 'path_to_geojson.geojson'
epsg_code_clu = get_geojson_epsg(sample_clu_path)
print("CLU EPSG code:", epsg_code_clu)

Parcel ID EPSG code: 4326
CLU EPSG code: 4326


In [5]:
#reproject Parcel ID geojsons to CONUS Albers Equal Area
def reproject_geojson(sample_parcels_path, sample_parcels_path_reproj, conus_epsg):
    # Read the GeoJSON file
    gdf = gpd.read_file(sample_parcels_path)
    
    # Reproject to the target CRS
    gdf_reprojected = gdf.to_crs(conus_epsg)
    
    # Save the reprojected GeoJSON file
    gdf_reprojected.to_file(sample_parcels_path_reproj, driver='GeoJSON')
    print(f"Reprojected GeoJSON file saved to '{sample_parcels_path_reproj}'.")

reproject_geojson(sample_parcels_path, sample_parcels_path_reproj, conus_epsg)

#reproject CLU geojsons to CONUS Albers Equal Area
def reproject_geojson(sample_clu_path, sample_clu_path_reproj, conus_epsg):
    # Read the GeoJSON file
    gdf = gpd.read_file(sample_clu_path)
    
    # Reproject to the target CRS
    gdf_reprojected = gdf.to_crs(conus_epsg)
    
    # Save the reprojected GeoJSON file
    gdf_reprojected.to_file(sample_clu_path_reproj, driver='GeoJSON')
    print(f"Reprojected GeoJSON file saved to '{sample_clu_path_reproj}'.")

reproject_geojson(sample_clu_path, sample_clu_path_reproj, conus_epsg)

/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Reprojected GeoJSON file saved to '/Users/sorenrundquist/Documents/ParcelCheck/test_parcel_id_reproj.geojson'.
Reprojected GeoJSON file saved to '/Users/sorenrundquist/Documents/ParcelCheck/test_clu_reproj.geojson'.


/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [6]:
#Add unique ID for CLUs
def add_column_shapefile(sample_clu_path_reproj, clu_id):
    # Read the shapefile
    gdf = gpd.read_file(sample_clu_path_reproj)
    
    # Add a new column with default value
    gdf[clu_id] = None
    
    # Save the updated shapefile
    gdf.to_file(sample_clu_path_reproj)
    print(f"New column '{clu_id}' added to the shapefile.")

def update_column_with_unique_id(sample_clu_path_reproj, clu_id):
    # Read the shapefile
    gdf = gpd.read_file(sample_clu_path_reproj)
    
    # Generate unique IDs
    gdf[clu_id] = range(1, len(gdf) + 1)
    
    # Save the updated shapefile
    gdf.to_file(sample_clu_path_reproj)
    print(f"{clu_id} updated with unique IDs.")

add_column_shapefile(sample_clu_path_reproj, clu_id)
update_column_with_unique_id(sample_clu_path_reproj, clu_id)

#Add unique ID for Parcel ID
def add_column_shapefile(sample_parcels_path_reproj, parcel_id):
    # Read the shapefile
    gdf = gpd.read_file(sample_parcels_path_reproj)
    
    # Add a new column with default value
    gdf[parcel_id] = None
    
    # Save the updated shapefile
    gdf.to_file(sample_parcels_path_reproj)
    print(f"New column '{parcel_id}' added to the shapefile.")

def update_column_with_unique_id(sample_parcels_path_reproj, parcel_id):
    # Read the shapefile
    gdf = gpd.read_file(sample_parcels_path_reproj)
    
    # Generate unique IDs
    gdf[parcel_id] = range(1, len(gdf) + 1)
    
    # Save the updated shapefile
    gdf.to_file(sample_parcels_path_reproj)
    print(f"{parcel_id} updated with unique IDs.")

add_column_shapefile(sample_parcels_path_reproj, parcel_id)
update_column_with_unique_id(sample_parcels_path_reproj, parcel_id)

/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


New column 'cluID' added to the shapefile.
cluID updated with unique IDs.


/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


New column 'parID' added to the shapefile.
parID updated with unique IDs.


/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [7]:
#Calculate area for parcels
def calculate_area(gdf):
    # Calculate area in square meters
    gdf[parcel_area] = gdf.geometry.area * 0.000247105
    return gdf

# Example usage:
if __name__ == "__main__":
    # Read your GeoDataFrame from a file or any other source
    gdf = gpd.read_file(sample_parcels_path_reproj)

    # Call the function to calculate area and add it as a new column
    gdf = calculate_area(gdf)

    # Print the first few rows to verify
    print(gdf.head())

    # Save the updated GeoDataFrame to a file or use it further in your analysis
    gdf.to_file(sample_parcels_path_reproj, driver='GeoJSON')

#Calculate area for CLUs
def calculate_area(gdf):
    # Calculate area in square meters
    gdf[clu_area] = gdf.geometry.area * 0.000247105
    return gdf

# Example usage:
if __name__ == "__main__":
    # Read your GeoDataFrame from a file or any other source
    gdf = gpd.read_file(sample_clu_path_reproj)

    # Call the function to calculate area and add it as a new column
    gdf = calculate_area(gdf) 

    # Print the first few rows to verify
    print(gdf.head())

    # Save the updated GeoDataFrame to a file or use it further in your analysis
    gdf.to_file(sample_clu_path_reproj, driver='GeoJSON')

   FID  parID                                           geometry    parArea
0    1      1  MULTIPOLYGON (((312225.744 2222608.950, 312232...   5.810520
1    3      2  MULTIPOLYGON (((312003.554 2222690.349, 312017...  15.734414
2    4      3  MULTIPOLYGON (((315566.435 2222815.336, 315569...  17.413717
3    5      4  MULTIPOLYGON (((315937.935 2222830.558, 315941...  50.785550
4    7      5  MULTIPOLYGON (((314991.587 2222776.751, 315049...  31.747429
   CLUCLSCD     layer  area     ID  _majority  cluID  \
0         2  IA_Merge    81  20797        1.0      1   
1         2  IA_Merge    73  20799        1.0      2   
2         2  IA_Merge    40  20801        1.0      3   
3         2  IA_Merge    50  20856        1.0      4   
4         2  IA_Merge    80  21576        1.0      5   

                                            geometry    cluArea  
0  MULTIPOLYGON (((312383.240 2220242.972, 312555...  80.893610  
1  MULTIPOLYGON (((312422.250 2219439.122, 311766...  73.345969  
2  MULTIP

/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [8]:
#Union the Parcel ID and CLU sample files
def union_geojsons(sample_parcels_path_reproj, sample_clu_path_reproj, union_results_path):
    # Read the GeoJSON files
    gdf1 = gpd.read_file(sample_parcels_path_reproj)
    gdf2 = gpd.read_file(sample_clu_path_reproj)
    
    # Union the GeoDataFrames
    union_gdf = gpd.overlay(gdf1, gdf2, how='union')
    
    # Save the result to a new GeoJSON file
    union_gdf.to_file(union_results_path , driver='GeoJSON')
    print(f"Union of '{sample_parcels_path_reproj}' and '{sample_clu_path_reproj}' saved to '{union_results_path}'.")

union_geojsons(sample_parcels_path_reproj, sample_clu_path_reproj, union_results_path)

/var/folders/nx/7596x84x0g1_qn0bk3krnnzr0000gn/T/ipykernel_67994/3990652312.py:8: UserWarning: `keep_geom_type=True` in overlay resulted in 17 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  union_gdf = gpd.overlay(gdf1, gdf2, how='union')
/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Union of '/Users/sorenrundquist/Documents/ParcelCheck/test_parcel_id_reproj.geojson' and '/Users/sorenrundquist/Documents/ParcelCheck/test_clu_reproj.geojson' saved to '/Users/sorenrundquist/Documents/ParcelCheck/parcel_CLU_union.geojson'.


In [9]:
#Calculate area (in acres) for union - or overlap
def calculate_area(gdf):
    # Calculate area in square meters
    gdf[union_area] = gdf.geometry.area * 0.000247105
    return gdf

# Example usage:
if __name__ == "__main__":
    # Read your GeoDataFrame from a file or any other source
    gdf = gpd.read_file(union_results_path)

    # Call the function to calculate area and add it as a new column
    gdf = calculate_area(gdf) 

    # Print the first few rows to verify
    print(gdf.head())

    # Save the updated GeoDataFrame to a file or use it further in your analysis
    gdf.to_file(union_results_path, driver='GeoJSON')

    FID  parID    parArea  CLUCLSCD     layer   area       ID  _majority  \
0   1.0    1.0   5.810520       2.0  IA_Merge  267.0  27703.0        1.0   
1  14.0   12.0  30.729257       2.0  IA_Merge  267.0  27703.0        1.0   
2   4.0    3.0  17.413717       2.0  IA_Merge   13.0  37136.0        1.0   
3   9.0    7.0  23.222999       2.0  IA_Merge   13.0  37136.0        1.0   
4   4.0    3.0  17.413717       2.0  IA_Merge   70.0  37150.0        1.0   

   cluID     cluArea                                           geometry  \
0   18.0  267.022727  POLYGON ((312321.850 2222668.088, 312325.695 2...   
1   18.0  267.022727  MULTIPOLYGON (((312737.814 2222037.099, 312737...   
2   80.0   12.948656  POLYGON ((315547.768 2222558.438, 315542.894 2...   
3   80.0   12.948656  POLYGON ((315415.004 2222593.181, 315437.063 2...   
4   83.0   70.166181  POLYGON ((315721.475 2222414.886, 315707.111 2...   

    uniArea  
0  0.019863  
1  0.095679  
2  0.050229  
3  7.069908  
4  0.258394  


/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [10]:
#Calculate the difference (in acres) between CLU area and union area
def calculate_dif(gdf):
    # Calculate area in square meters
    gdf[clu_area_minus_uni_area] = gdf[clu_area] - gdf[union_area]
    return gdf

# Example usage:
if __name__ == "__main__":
    # Read your GeoDataFrame from a file or any other source
    gdf = gpd.read_file(union_results_path)

    # Call the function to calculate area and add it as a new column
    gdf = calculate_dif(gdf) 

    # Print the first few rows to verify
    print(gdf.head())

    # Save the updated GeoDataFrame to a file or use it further in your analysis
    gdf.to_file(union_results_path, driver='GeoJSON')

    FID  parID    parArea  CLUCLSCD     layer   area       ID  _majority  \
0   1.0    1.0   5.810520       2.0  IA_Merge  267.0  27703.0        1.0   
1  14.0   12.0  30.729257       2.0  IA_Merge  267.0  27703.0        1.0   
2   4.0    3.0  17.413717       2.0  IA_Merge   13.0  37136.0        1.0   
3   9.0    7.0  23.222999       2.0  IA_Merge   13.0  37136.0        1.0   
4   4.0    3.0  17.413717       2.0  IA_Merge   70.0  37150.0        1.0   

   cluID     cluArea   uniArea  \
0   18.0  267.022727  0.019863   
1   18.0  267.022727  0.095679   
2   80.0   12.948656  0.050229   
3   80.0   12.948656  7.069908   
4   83.0   70.166181  0.258394   

                                            geometry  difOverlap  
0  POLYGON ((312321.850 2222668.088, 312325.695 2...  267.002864  
1  MULTIPOLYGON (((312737.814 2222037.099, 312737...  266.927048  
2  POLYGON ((315547.768 2222558.438, 315542.894 2...   12.898427  
3  POLYGON ((315415.004 2222593.181, 315437.063 2...    5.878747  
4  

/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [11]:
#Calculate statistics for overlapping polygons not null values greater than 5 acres (this gets rid of slivers)
def calculate_statistics_for_not_null_values(union_results_path, parcel_area, clu_area):
    # Read the GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(union_results_path)

    # Select rows where both parArea and cluArea are not null
    selected_gdf = gdf[(~gdf[parcel_area].isnull()) & (~gdf[clu_area].isnull())]

    selected_gdf2 = selected_gdf[selected_gdf.eval(expression)]

    # Calculate statistics
    count_val = selected_gdf2[clu_area_minus_uni_area].count()
    min_val = selected_gdf2[clu_area_minus_uni_area].min()
    max_val = selected_gdf2[clu_area_minus_uni_area].max()
    mean_val = selected_gdf2[clu_area_minus_uni_area].mean()
    std_val = selected_gdf2[clu_area_minus_uni_area].std()
    med_val = selected_gdf2[clu_area_minus_uni_area].median()

    return count_val, min_val, max_val, mean_val, std_val, med_val

count_val, min_val, max_val, mean_val, std_val, med_val = calculate_statistics_for_not_null_values(union_results_path, parcel_area, clu_area)
print("CLU-Parcel Overlap Count: %.0f" % count_val, "features")
print("CLU-Parcel Overlap Minimum Difference:: %.2f" % min_val, "acres")
print("CLU-Parcel Overlap Maximum Difference: %.2f" % max_val, "acres")
print("CLU-Parcel Overlap Mean Difference: %.2f" % mean_val, "acres")
print("CLU-Parcel Overlap Standard Deviation Difference: %.2f" % std_val, "acres")
print("CLU-Parcel Overlap Median Difference: %.2f" % med_val, "acres")

CLU-Parcel Overlap Count: 128 features
CLU-Parcel Overlap Minimum Difference:: 0.00 acres
CLU-Parcel Overlap Maximum Difference: 131.08 acres
CLU-Parcel Overlap Mean Difference: 21.34 acres
CLU-Parcel Overlap Standard Deviation Difference: 35.21 acres
CLU-Parcel Overlap Median Difference: 2.65 acres


In [12]:
#Calculate statistics for CLU polygons raw data
def calculate_stats_from_column(sample_clu_path_reproj, clu_area):
    # Read GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(sample_clu_path_reproj)

    # Calculate statistics from the specified column
    count_val = gdf[clu_area].count()
    min_val = gdf[clu_area].min()
    max_val = gdf[clu_area].max()
    mean_val = gdf[clu_area].mean()
    std_val = gdf[clu_area].std()
    med_val = gdf[clu_area].median()

    return count_val, min_val, max_val, mean_val, std_val, med_val

count_val, min_val, max_val, mean_val, std_val, med_val = calculate_stats_from_column(sample_clu_path_reproj, clu_area)
print("CLU Count: %.0f" % count_val, "features")
print("CLU Minimum: %.2f" % min_val, "acres")
print("CLU Maximum: %.2f" % max_val, "acres")
print("CLU Mean: %.2f" % mean_val, "acres")
print("CLU Standard Deviation: %.2f" % std_val, "acres")
print("CLU Median Deviation: %.2f" % med_val, "acres")

CLU Count: 139 features
CLU Minimum: 4.60 acres
CLU Maximum: 267.02 acres
CLU Mean: 51.45 acres
CLU Standard Deviation: 46.85 acres
CLU Median Deviation: 37.38 acres


In [13]:
#Calculate statistics for Parcel ID polygons raw data
def calculate_stats_from_column(sample_parcels_path_reproj, parcel_area):
    # Read GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(sample_parcels_path_reproj)

    # Calculate statistics from the specified column
    count_val = gdf[parcel_area].count()
    min_val = gdf[parcel_area].min()
    max_val = gdf[parcel_area].max()
    mean_val = gdf[parcel_area].mean()
    std_val = gdf[parcel_area].std()
    med_val = gdf[parcel_area].median()

    return count_val, min_val, max_val, mean_val, std_val, med_val

count_val, min_val, max_val, mean_val, std_val, med_val = calculate_stats_from_column(sample_parcels_path_reproj, parcel_area)
print("Parcel ID Count: %.0f" % count_val, "features")
print("Parcel ID Minimum:: %.2f" % min_val, "acres")
print("Parcel ID Maximum: %.2f" % max_val, "acres")
print("Parcel ID Mean: %.2f" % mean_val, "acres")
print("Parcel ID Standard Deviation: %.2f" % std_val, "acres")
print("Parcel ID Median Deviation: %.2f" % med_val, "acres")

Parcel ID Count: 121 features
Parcel ID Minimum:: 4.73 acres
Parcel ID Maximum: 158.18 acres
Parcel ID Mean: 47.52 acres
Parcel ID Standard Deviation: 38.67 acres
Parcel ID Median Deviation: 33.73 acres


In [14]:
#Calculate the minimum distance between centroids of Parcel ID and CLU polygons
def find_centroids(sample_parcels_path_reproj):
    # Read GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(sample_parcels_path_reproj)

    # Compute centroids
    centroids = gdf['geometry'].centroid

    return centroids

centroids1 = find_centroids(sample_parcels_path_reproj)
centroids2 = find_centroids(sample_clu_path_reproj)

# print("Centroids for GeoJSON 1:")
# print(centroids1)

# print("\nCentroids for GeoJSON 2:")
# print(centroids2)

def average_distance_to_nearest_neighbor(centroids1, centroids2):
    # Initialize list to store minimum distances
    min_distances = []

    # Compute pairwise distances between centroids
    for centroid1 in centroids1:
        distances = [distance.euclidean(centroid1, centroid2) for centroid2 in centroids2]
        min_distances.append(min(distances))

    # Compute the average of the minimum distances
    avg_distance = np.mean(min_distances)

    return avg_distance
def min_distance_to_nearest_neighbor(centroids1, centroids2):
    # Initialize list to store minimum distances
    min_distances = []

    # Compute pairwise distances between centroids
    for centroid1 in centroids1:
        distances = [distance.euclidean(centroid1, centroid2) for centroid2 in centroids2]
        min_distances.append(min(distances))

    # Compute the min of the minimum distances
    min_distance = np.min(min_distances)

    return min_distance
def max_distance_to_nearest_neighbor(centroids1, centroids2):
    # Initialize list to store minimum distances
    min_distances = []

    # Compute pairwise distances between centroids
    for centroid1 in centroids1:
        distances = [distance.euclidean(centroid1, centroid2) for centroid2 in centroids2]
        min_distances.append(min(distances))

    # Compute the max of the minimum distances
    max_distance = np.max(min_distances)

    return max_distance
def std_distance_to_nearest_neighbor(centroids1, centroids2):
    # Initialize list to store minimum distances
    min_distances = []

    # Compute pairwise distances between centroids
    for centroid1 in centroids1:
        distances = [distance.euclidean(centroid1, centroid2) for centroid2 in centroids2]
        min_distances.append(min(distances))

    # Compute the std of the minimum distances
    std_distance = np.std(min_distances)

    return std_distance
avg_distance = average_distance_to_nearest_neighbor(centroids1, centroids2)
min_distance = min_distance_to_nearest_neighbor(centroids1, centroids2)
max_distance = max_distance_to_nearest_neighbor(centroids1, centroids2)
std_distance = std_distance_to_nearest_neighbor(centroids1, centroids2)
print("Mean distance to nearest neighbor: %.2f" % avg_distance, "meters")
print("Max distance to nearest neighbor: %.2f" % max_distance, "meters")
print("Min distance to nearest neighbor: %.2f" % min_distance, "meters")
print("STD distance to nearest neighbor: %.2f" % std_distance, "meters")

/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/scipy/spatial/distance.py:463: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  u = _validate_vector(u)
/Users/sorenrundquist/Library/Python/3.10/lib/python/site-packages/scipy/spatial/distance.py:464: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  v = _validate_vector(v)


Mean distance to nearest neighbor: 118.74 meters
Max distance to nearest neighbor: 649.50 meters
Min distance to nearest neighbor: 0.69 meters
STD distance to nearest neighbor: 122.42 meters
